This is the last assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

Just execute all cells one after the other and you are done - just note that in the last one you should update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"

The purpose of this assignment is to learn how feature engineering boosts model performance. You will apply Discrete Fourier Transformation on the accelerometer sensor time series and therefore transforming the dataset from the time to the frequency domain. 

After that, you’ll use a classification algorithm of your choice to create a model and submit the new predictions to the grader. Done.


In [0]:
credentials_1 = {'password':"""*****""",
                 'custom_url':'****',
                 'username':'******' }



Let's create a SparkSession object and put the Cloudant credentials into it

In [0]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",credentials_1['custom_url'].split('@')[1])\
    .config("cloudant.username", credentials_1['username'])\
    .config("cloudant.password",credentials_1['password'])\
    .getOrCreate()

Now it’s time to read the sensor data and create a temporary query table.

In [0]:
df=spark.read.load('shake_classification', "org.apache.bahir.cloudant")
df.createOrReplaceTempView("df")

We need to make sure SystemML is installed.


In [0]:
!pip install systemml

    100% |████████████████████████████████| 9.7MB 4.8MB/s eta 0:00:01    38% |████████████▍                   | 3.8MB 59.5MB/s eta 0:00:01
  Running setup.py bdist_wheel for systemml ... done
  Stored in directory: /gpfs/fs01/user/s21d-5d99ed86472532-829ddaa28d8f/.cache/pip/wheels/cf/07/79/b3ed6f12afe06b2ab55d60dcfd62e66240f5d8c6088a518177
Successfully built systemml
notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


We’ll use Apache SystemML to implement Discrete Fourier Transformation. This way all computation continues to happen on the Apache Spark cluster for advanced scalability and performance.

In [0]:
from systemml import MLContext, dml
ml = MLContext(spark)

As you’ve learned from the lecture, implementing Discrete Fourier Transformation in a linear algebra programming language is simple. Apache SystemML DML is such a language and as you can see the implementation is straightforward and doesn’t differ too much from the mathematical definition (Just note that the sum operator has been swapped with a vector dot product using the %*% syntax borrowed from R
):

<img style="float: left;" src="https://wikimedia.org/api/rest_v1/media/math/render/svg/1af0a78dc50bbf118ab6bd4c4dcc3c4ff8502223">



In [0]:
dml_script = '''
PI = 3.141592654
N = nrow(signal)

n = seq(0, N-1, 1)
k = seq(0, N-1, 1)

M = (n %*% t(k))*(2*PI/N)

Xa = cos(M) %*% signal
Xb = sin(M) %*% signal

DFT = cbind(Xa, Xb)
'''

Now it’s time to create a function which takes a single row Apache Spark data frame as argument (the one containing the accelerometer measurement time series for one axis) and returns the Fourier transformation of it. In addition, we are adding an index column for later joining all axis together and renaming the columns to appropriate names. The result of this function is an Apache Spark DataFrame containing the Fourier Transformation of its input in two columns. 


In [0]:
from pyspark.sql.functions import monotonically_increasing_id

def dft_systemml(signal,name):
    prog = dml(dml_script).input('signal', signal).output('DFT')
    
    return (

    #execute the script inside the SystemML engine running on top of Apache Spark
    ml.execute(prog) 
     
         #read result from SystemML execution back as SystemML Matrix
        .get('DFT') 
     
         #convert SystemML Matrix to ApacheSpark DataFrame 
        .toDF() 
     
         #rename default column names
        .selectExpr('C1 as %sa' % (name), 'C2 as %sb' % (name)) 
     
         #add unique ID per row for later joining
        .withColumn("id", monotonically_increasing_id())
    )
        




Now it’s time to create DataFrames containing for each accelerometer sensor axis and one for each class. This means you’ll get 6 DataFrames. Please implement this using the relational API of DataFrames or SparkSQL.


In [0]:
x0 = spark.sql("SELECT X from df where class = 0")
y0 = spark.sql("SELECT Y from df where class = 0")
z0 = spark.sql("SELECT Z from df where class = 0")
x1 = spark.sql("SELECT X from df where class = 1")
y1 = spark.sql("SELECT Y from df where class = 1")
z1 = spark.sql("SELECT Z from df where class = 1")

Since we’ve created this cool DFT function before, we can just call it for each of the 6 DataFrames now. And since the result of this function call is a DataFrame again we can use the pyspark best practice in simply calling methods on it sequentially. So what we are doing is the following:

- Calling DFT for each class and accelerometer sensor axis.
- Joining them together on the ID column. 
- Re-adding a column containing the class index.
- Stacking both Dataframes for each classes together



In [0]:
from pyspark.sql.functions import lit

df_class_0 = dft_systemml(x0,'x') \
    .join(dft_systemml(y0,'y'), on=['id'], how='inner') \
    .join(dft_systemml(z0,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(0))
    
df_class_1 = dft_systemml(x1,'x') \
    .join(dft_systemml(y1,'y'), on=['id'], how='inner') \
    .join(dft_systemml(z1,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(1))

df_dft = df_class_0.union(df_class_1)

df_dft.show()

[Stage 3:=================>                                        (3 + 5) / 10]
[Stage 3:==================================>                       (6 + 4) / 10]
[Stage 3:====================================================>     (9 + 1) / 10]
SystemML Statistics:
Total execution time:		4.170 sec.
Number of executed Spark inst:	0.

                                                                                

[Stage 7:>                                                         (0 + 5) / 10]
[Stage 7:===========>                                              (2 + 6) / 10]
[Stage 7:=============================>                            (5 + 5) / 10]
[Stage 7:========================================>                 (7 + 3) / 10]
SystemML Statistics:
Total execution time:		4.709 sec.
Number of executed Spark inst:	0.

                                                                                

[Stage 11:>                                                        (0 + 5) / 10]
[Stage 1

Please create a VectorAssembler which consumes the newly created DFT columns and produces a column “features”


In [0]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(
    inputCols=["xa", "xb", "ya", "yb", "za", "zb"],
    outputCol="features")

Please insatiate a classifier from the SparkML package and assign it to the classifier variable. Make sure to set the “class” column as target.


In [0]:
from pyspark.ml.classification import RandomForestClassifier
classifier = RandomForestClassifier(labelCol="class", featuresCol="features", numTrees=10)

Let’s train and evaluate…


In [0]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [0]:
model = pipeline.fit(df_dft)

In [0]:
prediction = model.transform(df_dft)

In [0]:
prediction.show()

+-----------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----+--------------------+--------------------+--------------------+----------+
|         id|                 xa|                 xb|                 ya|                 yb|                 za|                 zb|class|            features|       rawPrediction|         probability|prediction|
+-----------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----+--------------------+--------------------+--------------------+----------+
|         26| 380.66434005495194|-137.07822473828313| -139.3470983812975|  22.67267886239656|-247.93697521077704|-15.169114611846743|    0|[380.664340054951...|          [10.0,0.0]|           [1.0,0.0]|       0.0|
|         29| 104.74324299209692| -92.43964309825088| -32.27421440203938|  39.39162630938283|-25.105013725863852|  95.46710868338275|    0|[104.

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("class")
    
binEval.evaluate(prediction) 

1.0

If you are happy with the result (I’m happy with > 0.8) please submit your solution to the grader by executing the following cells, please don’t forget to obtain an assignment submission token (secret) from the Courera’s graders web page and paste it to the “secret” variable below, including your email address you’ve used for Coursera. 


In [0]:
!rm -Rf a2_m4.parquet

In [0]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m4.json')

In [0]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2019-02-11 14:22:33--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2.2K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,289       --.-K/s   in 0s      

2019-02-11 14:22:33 (13.8 MB/s) - ‘rklib.py’ saved [2289/2289]



In [0]:
!zip -r a2_m4.json.zip a2_m4.json

  adding: a2_m4.json/ (stored 0%)
  adding: a2_m4.json/_SUCCESS (stored 0%)
  adding: a2_m4.json/._SUCCESS.crc (stored 0%)
  adding: a2_m4.json/part-00000-e6ed1da6-f4b2-4a83-a305-1395dcb05426.json (deflated 80%)
  adding: a2_m4.json/.part-00000-e6ed1da6-f4b2-4a83-a305-1395dcb05426.json.crc (stored 0%)


In [0]:
!base64 a2_m4.json.zip > a2_m4.json.zip.base64

In [0]:
from rklib import submit
key = "******"
part = "*****"
email = "annmargaret.tutu@icloud.com" #"romeo.kienzler@ch.ibm.com"
secret = "*****"

with open('a2_m4.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"B8wXV","id":"f_F-qCtuEei_fRLwaVDk3g~B8wXV~ztyEui46Eempogq4ugoMgA","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
